In [42]:
from dask import dataframe as dd
from dask_ml.linear_model import LinearRegression
from dask_ml.preprocessing import Categorizer, DummyEncoder
from sklearn.pipeline import Pipeline, make_pipeline

In [43]:
ddf=dd.read_csv(
    "https://gist.githubusercontent.com/geraldwal/b5a83f4c670abe0a662abce558e5d433/raw/bce4bbfc63355606e4503964e25798b5d2190b9b/hour%2520-%2520Python%2520Bike%2520Sharing",
    sep=",",
    parse_dates=["dteday"])

In [44]:
ddf = ddf.rename(
    columns={
        "weathersit": "weather",
        "mnth": "month",
        "hr": "hour",
        "hum": "humidity",
        "cnt": "count",
        "yr": "year",
    })

In [45]:
cats= ["season","month","hour","holiday","weekday","workingday","weather"]

pipeline = make_pipeline(Categorizer(columns=cats), DummyEncoder(columns=cats))
ddf_onehot=pipeline.fit_transform(ddf)


In [46]:
def split_data(dataset, Target):
    X = dataset.loc[:, dataset.columns != Target]
    y = dataset.loc[:, Target]
    train_size = int(len(dataset) * 0.875)
    X_train, X_test, y_train, y_test = (
        X.loc[0:train_size], #dask uses loc instead of iloc 
        X.loc[train_size : len(dataset)],
        y.loc[0:train_size],
        y.loc[train_size : len(dataset)],
    )
    return X_train, X_test, y_train, y_test

In [47]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = split_data(ddf_onehot, "registered")
x_train_casual, x_test_casual, y_train_casual, y_test_casual = split_data(ddf_onehot, "casual")
x_train_count, x_test_count, y_train_count, y_test_count = split_data(ddf_onehot, "count")

In [48]:
def score_lin(X_train, X_test, y_train, y_test):
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    print("Intercept:", lm.intercept_)
    print("Coefficients:", lm.coef_)
    print("Mean squared error (MSE): {:.2f}".format(mean_squared_error(y_test, y_pred)))
    print("Variance score (R2): {:.2f}".format(r2_score(y_test, y_pred)))
    return y_pred

In [49]:
train_dropped_target = x_train_count.drop(["casual", "registered"], axis = 1)
test_dropped_target = x_test_count.drop(["casual", "registered"], axis = 1)
baseline_registered_count = score_lin(train_dropped_target, test_dropped_target, y_train_count, y_test_count)
baseline_registered_count = score_lin(x_train_count_arr, x_test_count_arr, y_train_count_arr, y_test_count_arr)

TypeError: This estimator does not support dask dataframes.